In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
from torch.utils.data import Dataset, DataLoader
import glob
import wandb
import os
import torch.optim as optimizers
import dfs_code
from torch_geometric.data import InMemoryDataset, Data
import pickle
import torch
import torch.nn as nn
import tqdm
import copy
import pandas as pd
import torch.nn.functional as F
import sys
import yaml
import functools
from ml_collections import ConfigDict
from ogb.graphproppred import PygGraphPropPredDataset

sys.path = ['../../src'] + sys.path
from dfs_transformer import DFSCodeSeq2SeqFC, Deepchem2TorchGeometric, Trainer, to_cuda

Using backend: pytorch


In [3]:
fname = '../../config/selfattn/finetune_ogb.yaml'
with open(fname) as file:
    config = ConfigDict(yaml.load(file, Loader=yaml.FullLoader))

In [4]:
config

accumulate_grads: 2
batch_size: 50
clip_gradient: 0.5
decay_factor: 0.8
es_improvement: 0.0
es_path: null
es_patience: 10
fingerprint: cls
gpu_id: 0
load_last: true
lr: 0.0003
lr_head: 0.003
lr_patience: 3
lr_pretrained: 0.0003
minimal_lr: 6.0e-08
n_classes: 349
n_epochs: 25
n_frozen: 5
path: ../../results/ogbn_mag/timeout1/
pretrained_class: DFSCodeSeq2SeqFC
pretrained_dir: null
pretrained_entity: dfstransformer
pretrained_model: rnd2min
pretrained_project: ogbn-mag
pretrained_yaml: null
require_min_dfs_code: false
seed: 123
strict: true
use_local_yaml: false
weight_decay: 0.1

In [5]:
config.pretrained_project = 'pubchem'
config.pretrained_model = 'rnd2min2-10M-euler'
config.es_period = 300
config.lr = 0.000003 # 0.000003 war gut
config.alpha = 0

In [6]:
config.require_min_dfs_code = False

In [7]:
onlyRandom = not config.require_min_dfs_code

In [8]:
mol_csv = pd.read_csv('../../datasets/ogbg_molhiv/mol.csv')

dataset = PygGraphPropPredDataset(name = "ogbg-molhiv") 
split_idx = dataset.get_idx_split() 

# check whether we get the correct splits

In [9]:
for split in ["train", "valid", "test"]:
    csv_labels = mol_csv["HIV_active"][split_idx[split].numpy()].to_numpy()
    ogb_labels = np.asarray([d.y.item() for d in dataset[split_idx[split]]])
    if (ogb_labels == csv_labels).sum() == len(ogb_labels):
        print("All %s labels are identical."%split)

All train labels are identical.
All valid labels are identical.
All test labels are identical.


In [10]:
train_smiles = mol_csv["smiles"][split_idx["train"].numpy()].to_numpy()
train_labels = mol_csv["HIV_active"][split_idx["train"].numpy()].to_numpy()
valid_smiles = mol_csv["smiles"][split_idx["valid"].numpy()].to_numpy()
valid_labels = mol_csv["HIV_active"][split_idx["valid"].numpy()].to_numpy()
test_smiles = mol_csv["smiles"][split_idx["test"].numpy()].to_numpy()
test_labels = mol_csv["HIV_active"][split_idx["test"].numpy()].to_numpy()

In [11]:
loaddir = "../../results/mymoleculenet_plus_features/hiv/1/" # ogbg uses other smiles than deepchem...
loaddir = None
train = Deepchem2TorchGeometric(train_smiles, train_labels, loaddir=loaddir, onlyRandom=onlyRandom)
valid = Deepchem2TorchGeometric(valid_smiles, valid_labels, loaddir=loaddir, onlyRandom=onlyRandom)
test = Deepchem2TorchGeometric(test_smiles, test_labels, loaddir=loaddir, onlyRandom=onlyRandom)

In [12]:
def collate_fn(dlist, alpha=config.alpha):
    node_batch = [] 
    edge_batch = []
    y_batch = []
    rnd_code_batch = []
    for d in dlist:
        node_batch += [d.node_features.clone()]
        edge_batch += [d.edge_features.clone()]
        rnd_code, rnd_index = dfs_code.rnd_dfs_code_from_torch_geometric(d, d.z.numpy().tolist(), 
                                                                         np.argmax(d.edge_attr.numpy(), axis=1).tolist())
        rnd_code = torch.tensor(np.asarray(rnd_code), dtype=torch.long)
        rnd_code_batch += [rnd_code]
        y_batch += [d.y.clone()]
    y = torch.cat(y_batch).unsqueeze(1)
    y = (1-alpha)*y + alpha/2
    return rnd_code_batch, node_batch, edge_batch, y

In [13]:
coll_val = functools.partial(collate_fn, alpha=0)

In [14]:
trainloader = DataLoader(train, shuffle=True, batch_size=config.batch_size, collate_fn=collate_fn, num_workers=8)
validloader = DataLoader(valid, shuffle=False, batch_size=config.batch_size, collate_fn=coll_val, num_workers=8)
testloader = DataLoader(test, shuffle=False, batch_size=config.batch_size, collate_fn=coll_val, num_workers=8)

In [15]:
name = "rnd2min2-10M-euler-labelsmoothing"
mode = "online"

In [16]:
# download pretrained model
run = wandb.init(mode=mode, 
                 project=config.pretrained_project, 
                 entity=config.pretrained_entity, 
                 job_type="inference")
model_at = run.use_artifact(config.pretrained_model + ":latest")
model_dir = model_at.download()
run.finish()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: chrisxx (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-09-24 11:35:42.072818: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt

wandb: Downloading large artifact rnd2min2-10M-euler:latest, 95.63MB. 2 files... Done. 0:0:0


In [17]:
with open(model_dir+"/config.yaml") as file:
    mconfig = ConfigDict(yaml.load(file, Loader=yaml.FullLoader))

In [18]:
config.model = mconfig

In [19]:
run = wandb.init(mode=mode, project="ogbg-hiv", entity="dfstransformer", 
                 name=name, config=config.to_dict(), job_type="evaluation")

wandb: wandb version 0.12.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
2021-09-24 11:35:48.753272: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/cuda/extras/CUPTI/lib64/:/opt/intel/lib:/opt/intel/mkl/lib/intel64:/opt/intel:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/bin/x86-64_linux:/opt/ibm/ILOG/CPLEX_Studio1210/cplex/python/3.7/x86-64_linux:/opt/intel/clck_latest/lib:/opt/intel/daal/lib:/opt/intel/intelpython3/lib:/opt/intel/ipp/lib:/opt/intel/itac_2019/lib:/opt/intel/itac_latest/lib:/opt/intel/mkl/lib:/opt/intel/mkl_/lib:/opt/intel/mpirt/lib:/opt/intel/tbb/lib:/opt/intel/clck/2019.0/lib:/opt/intel/compilers_and_libraries_2019/linux/lib:/opt/intel/compilers_and_libraries/linux/lib:/opt/intel/itac/2019.0.018/lib:/opt/intel/itac_2019/intel64/lib:/opt/intel/itac_latest/intel64/lib:/o

In [20]:
m = mconfig.model
t = config

In [21]:
ce = nn.CrossEntropyLoss(ignore_index=-1)
bce = nn.BCEWithLogitsLoss()    

In [22]:
class TransformerPlusHead(nn.Module):
    def __init__(self, encoder, n_encoding, n_classes, fingerprint='cls'):
        super(TransformerPlusHead, self).__init__()
        self.encoder = encoder
        self.head = nn.Linear(n_encoding, n_classes)
        self.fingerprint = fingerprint
    
    def forward(self, C, N, E):
        features = self.encoder.encode(C, N, E, method=self.fingerprint)
        output = self.head(features)
        return output
        

In [23]:
from ogb.graphproppred import Evaluator

evaluator = Evaluator(name = 'ogbg-molhiv')

In [24]:
print(evaluator.expected_input_format)
print(evaluator.expected_output_format)

==== Expected input format of Evaluator for ogbg-molhiv
{'y_true': y_true, 'y_pred': y_pred}
- y_true: numpy ndarray or torch tensor of shape (num_graph, num_task)
- y_pred: numpy ndarray or torch tensor of shape (num_graph, num_task)
where y_pred stores score values (for computing AUC score),
num_task is 1, and each row corresponds to one graph.
nan values in y_true are ignored during evaluation.

==== Expected output format of Evaluator for ogbg-molhiv
{'rocauc': rocauc}
- rocauc (float): ROC-AUC score averaged across 1 task(s)



In [25]:
def loss(pred, y, ce=bce):
    return ce(pred, y)

def acc(pred, y):
    y_pred = (pred > 0.5).squeeze()
    y = (y > 0.5)
    return (y_pred == y.squeeze()).sum()/len(y)
    

In [26]:
device = torch.device('cuda:%d'%t.gpu_id if torch.cuda.is_available()  else 'cpu')
encoder = DFSCodeSeq2SeqFC(**m)
    
if t.load_last and model_dir is not None:
    encoder.load_state_dict(torch.load(model_dir+'/checkpoint.pt', map_location=device))

In [27]:
model = TransformerPlusHead(encoder, m.emb_dim*5*m.n_class_tokens, 1, fingerprint=t.fingerprint)

In [28]:
param_groups = [
    {'amsgrad': False,
     'betas': (0.9,0.98),
     'eps': 1e-09,
     'lr': 0.000003,
     'params': model.encoder.parameters(),
     'weight_decay': 0},
    {'amsgrad': False,
     'betas': (0.9, 0.999),
     'eps': 1e-08,
     'lr': 0.001,
     'params': model.head.parameters(),
     'weight_decay': 0}
]

In [29]:
del t.model

In [30]:
t

accumulate_grads: 2
alpha: 0
batch_size: 50
clip_gradient: 0.5
decay_factor: 0.8
es_improvement: 0.0
es_path: null
es_patience: 10
es_period: 300
fingerprint: cls
gpu_id: 0
load_last: true
lr: 3.0e-06
lr_head: 0.003
lr_patience: 3
lr_pretrained: 0.0003
minimal_lr: 6.0e-08
n_classes: 349
n_epochs: 25
n_frozen: 5
path: ../../results/ogbn_mag/timeout1/
pretrained_class: DFSCodeSeq2SeqFC
pretrained_dir: null
pretrained_entity: dfstransformer
pretrained_model: rnd2min2-10M-euler
pretrained_project: pubchem
pretrained_yaml: null
require_min_dfs_code: false
seed: 123
strict: true
use_local_yaml: false
weight_decay: 0.1

In [31]:
trainer = Trainer(model, trainloader, loss, validloader=validloader, metrics={'acc': acc}, wandb_run = run, param_groups=param_groups, **t)

In [32]:
trainer.fit()

Valid 1: loss 0.179720 1.0000:  29%|█████████████████████████████████████████▎                                                                                                     | 24/83 [00:02<00:05, 10.97it/s]


Valid 1: loss 0.179720 0.9000:  58%|██████████████████████████████████████████████████████████████████████████████████▋                                                            | 48/83 [00:05<00:03,  9.99it/s]


Valid 1: loss 0.179720 1.0000:  88%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                 | 73/83 [00:08<00:01,  9.59it/s]


Valid 1: loss 0.161353 1.0000:  17%|████████████████████████                                                                                                                       | 14/83 [00:02<00:07,  9.02it/s]


Valid 1: loss 0.161353 0.9800:  49%|██████████████████████████████████████████████████████████████████████▋                                                                        | 41/83 [00:04<00:04,  8.71it/s]


...

Valid 22: loss 0.099762 1.0000:  30%|██████████████████████████████████████████▊                                                                                                   | 25/83 [00:03<00:06,  8.87it/s]


Valid 22: loss 0.099762 0.9600:  61%|███████████████████████████████████████████████████████████████████████████████████████▎                                                      | 51/83 [00:05<00:02, 10.86it/s]


Valid 22: loss 0.099762 0.9600:  92%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████            | 76/83 [00:07<00:00,  9.94it/s]


Epoch 22: loss 0.099762 0.9200:  85%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                     | 560/659 [02:12<00:23,  4.23it/s]

EarlyStopping counter: 10 out of 10


In [33]:
model.load_state_dict(torch.load(trainer.es_path+'checkpoint.pt'))

<All keys matched successfully>

In [34]:
def compute_roc(model, loader):
    with torch.no_grad():
        preds = []
        ys = []
        for i, data in tqdm.tqdm(enumerate(testloader)):
            data = [to_cuda(d, device) for d in data]
            pred = model(*data[:-1])
            preds += [pred.cpu()]
            ys += [data[-1].cpu()]
        preds = torch.cat(preds, dim=0)
        ys = torch.cat(ys, dim=0)
        return evaluator.eval({'y_true':ys, 'y_pred':preds})['rocauc']

In [35]:
run.log({'Valid ROCAUC': compute_roc(model, validloader)})
run.log({'Test ROCAUC': compute_roc(model, testloader)})

83it [00:07, 11.33it/s]
83it [00:07, 11.83it/s]


#store config and model
with open(t.es_path+'config.yaml', 'w') as f:
    yaml.dump(config.to_dict(), f, default_flow_style=False)
if name is not None and mode != "offline":
    trained_model_artifact = wandb.Artifact(name, type="model", description="trained selfattn model")
    trained_model_artifact.add_dir(t.es_path)
    run.log_artifact(trained_model_artifact)

In [36]:
exit()

batch-loss,0.25964
loss,0.09976
batch-acc,0.92
acc,0.97141
learning rate,0.0
_runtime,3494
_timestamp,1632479641
_step,14471
valid-loss,0.07398
valid-acc,0.98458
train-loss,0.09976


batch-loss,▃▂▄▂▁▅▂▂▄▃▁▃▃▁▄▃▂▂▅▁▂▃▂▃▂▄▄▁▅▂▃▂█▁▂▁▂▁▃▃
loss,█▆▅▅▄▅▅▄▄▅▄▄▄▄▄▄▄▄▅▄▃▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▁▄▃▄
batch-acc,▆▇▄▇█▄▆▇▅▆█▆▇█▄▆▆▅▃█▆▅▇▆▇▅▄█▄▇▇▇▁█▆█▇█▅▅
acc,▂▂▂▁▃▂▂▂▂▁▂▂▂▂▂▂▂▂▁▃▅▃▂▃▃▃▃▃▃▃▃▃▃▂▃▃█▃▄▃
learning rate,██████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
valid-loss,█▇▆▅▄▃▃▃▄▃▃▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▂▂▂▂▁▁▂▂
valid-acc,▁▁▁▂▂▂▃▄▄▄▅▆▅▆▅▆▄▆▆▅▆▆▅▇▆▆▅▆▇▆▅▇▆█▇▇▆▇▇▇
train-loss,█▅▅▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
